# EDA

In [1]:
import pandas as pd
import numpy as np

orders=pd.read_csv('olist_orders_dataset.csv')
items=pd.read_csv('olist_order_items_dataset.csv')
product=pd.read_csv('olist_products_dataset.csv')
cust=pd.read_csv('olist_customers_dataset.csv')
reviews=pd.read_csv('olist_order_reviews_dataset.csv', encoding='latin1')
payment=pd.read_csv('olist_order_payments_dataset.csv')
sellers = pd.read_csv("olist_sellers_dataset.csv")

In [2]:
#join data
orders=orders.merge(cust, how='left', left_on='customer_id', right_on='customer_id')
orders=orders.merge(reviews, how='left', left_on='order_id', right_on='order_id')
orders=orders.merge(payment, how='left', left_on='order_id', right_on='order_id')
items=items.merge(product, how='left', on='product_id')


items=items.merge(sellers, how='left', on='seller_id')
basedata=items.merge(orders, how='left', left_on='order_id', right_on='order_id')

# retain what we need
basedata=basedata.drop(['review_id',
                        'review_answer_timestamp',
                        'customer_zip_code_prefix',
                        'shipping_limit_date',
                        'seller_zip_code_prefix', 'seller_city', 'order_delivered_carrier_date',
                        'customer_city','review_comment_message', 'review_creation_date', 'review_comment_title',
                        'product_weight_g',
                        ], axis=1)

basedata['order_delivered_customer_date'] = pd.to_datetime(basedata['order_delivered_customer_date'], format = "%Y-%m-%d %H:%M:%S")
basedata['order_estimated_delivery_date'] = pd.to_datetime(basedata['order_estimated_delivery_date'], format = "%Y-%m-%d %H:%M:%S")

Feature Engineering

In [3]:
# add Time to receive from estimate & Time to review columns
basedata['Time to receive from estimate'] = (basedata['order_delivered_customer_date'] - basedata['order_estimated_delivery_date']).dt.total_seconds()

basedata['order_purchase_timestamp'] = pd.to_datetime(basedata['order_purchase_timestamp'], format = "%Y-%m-%d %H:%M:%S")
basedata['order_approved_at'] = pd.to_datetime(basedata['order_approved_at'],format = "%Y-%m-%d %H:%M:%S")
basedata.loc[:,'approv_duration'] = ( basedata['order_approved_at'] - basedata['order_purchase_timestamp'] ).dt.total_seconds()

#seller state column formation
seller_state_counts = basedata['seller_state'].value_counts()
basedata['countstate'] = basedata['seller_state'].map(seller_state_counts)
basedata['newseller_state'] = np.where(basedata['countstate']>1000, basedata['seller_state'],'others')

#cust state column formation
cust_state_counts = basedata['customer_state'].value_counts()
basedata['cust_countstate'] = basedata['customer_state'].map(cust_state_counts)
basedata['newcust_state'] = np.where(basedata['cust_countstate']>1000, basedata['customer_state'],'others')

#prod state column formation
prod_state_counts = basedata['product_category_name'].value_counts()
basedata['prod_count'] = basedata['product_category_name'].map(prod_state_counts)
basedata['newproduct_cat'] = np.where(basedata['prod_count']>7500, basedata['product_category_name'],'unspecified')

duplicates = basedata[basedata.duplicated(subset=['order_id', 'order_item_id'], keep=False)]

#calculating the total payment, total freight, and total price.
basedata2=basedata
basedata2=basedata2[~basedata2.duplicated(subset=['order_id','order_item_id', 'payment_sequential', 'payment_value'],keep=False)] #removing duplicates becaus of left joins
basedata2['total_payment']= basedata2.groupby(['order_id','order_item_id'])['payment_value'].transform('sum') #totaling the payment value
basedata2['total_price']= basedata2.groupby(['order_id'])['price'].transform('sum') #sum of price
basedata2['total_freight']= basedata2.groupby(['order_id'])['freight_value'].transform('sum') #sum of freight value

basedata2.loc[:,'payment_dif'] = basedata2['total_payment']-basedata2['total_price']-basedata2['total_freight'] #payment difference check
payment_dif_check2 = basedata2[basedata2['payment_dif'] > 1]

basedata2dupes=basedata[basedata.duplicated(subset=['order_id','order_item_id', 'payment_sequential', 'payment_value'],keep=False)] #dupes removal chekcing

#invalid delivery (has a delivery date but the status is not delivered)
invalid_deliv_rows = basedata2[(basedata2['order_status'] != 'delivered') & (basedata2['order_delivered_customer_date'].notnull())]
basedata2 = basedata2.drop(invalid_deliv_rows.index).reset_index(drop=True)

#has no reviews
no_reviews = basedata2[basedata2['review_score'].isnull()]

#calculating the difference between the estimated delivery date againts the delivered date
basedata2.loc[:,'estimated-delivered'] = basedata2['order_estimated_delivery_date'] - basedata2['order_delivered_customer_date']
basedata2.loc[:,'estimated-delivered-secs'] = basedata2.loc[:,'estimated-delivered'].dt.total_seconds()

#calculating the approval duration (purchase_timestamp - approved_at)
basedata2['order_purchase_timestamp'] = pd.to_datetime(basedata2['order_purchase_timestamp'], format = "%Y-%m-%d %H:%M:%S")
basedata2['order_approved_at'] = pd.to_datetime(basedata2['order_approved_at'],format = "%Y-%m-%d %H:%M:%S")
basedata2.loc[:,'approv_duration'] = ( basedata2['order_approved_at'] - basedata2['order_purchase_timestamp'] ).dt.total_seconds()

#calculating the freight ratio
basedata2.loc[:,'freight ratio'] = basedata2['total_freight']/(basedata2['total_price'])

#removing invalid deliveries, no reviews, and duplicates
basedata2 = basedata2.drop(invalid_deliv_rows.index).reset_index(drop=True)
basedata2 = basedata2.drop(no_reviews.index).reset_index(drop = True)
basedata2=basedata2[~basedata2.duplicated(subset=['order_id'],keep=False)] #removing duplicates becaus of left joins


/tmp/ipython-input-3-2323592359.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basedata2['total_payment']= basedata2.groupby(['order_id','order_item_id'])['payment_value'].transform('sum') #totaling the payment value
/tmp/ipython-input-3-2323592359.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basedata2['total_price']= basedata2.groupby(['order_id'])['price'].transform('sum') #sum of price
/tmp/ipython-input-3-2323592359.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

# Modelling

In [4]:
# for the model
df = basedata2[['order_status','review_score','Time to receive from estimate','approv_duration','total_payment', 'total_price','freight ratio','product_description_lenght',
'product_photos_qty','newproduct_cat' ]]

df = df[df['order_status'] == 'delivered']
df['review_score'] = np.where(df['review_score'].isin([4, 5]), 1,0)

df=df.drop('order_status', axis=1)
df[['product_description_lenght', 'product_photos_qty']] = df[['product_description_lenght', 'product_photos_qty']].fillna(0) #we assume NA means 0 as there is no 0 value for both desc and photo qty
df = df.dropna() #dropping the other NA values

#dummies product cat
df_dummies4 = pd.get_dummies(df['newproduct_cat'], prefix='productcat_',dtype="int",drop_first=True)
# Append the dummies back to the original DataFrame
df = pd.concat([df, df_dummies4], axis=1)
df=df.drop('newproduct_cat', axis=1)

/tmp/ipython-input-4-1924862702.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_score'] = np.where(df['review_score'].isin([4, 5]), 1,0)


Random Forest

In [ ]:
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from xgboost import XGBClassifier as XGB
from sklearn.metrics import precision_recall_fscore_support


y_value = df['review_score'] # set the y
y_values = np.ravel(y_value) # change to an array (list)

x_values = df.drop('review_score', axis=1) # drop the y from the dataframe

# split data into training and test
from sklearn.model_selection  import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_values, y_value, test_size = 0.2, random_state=4567, stratify=y_value)

# print the shapes to check everything is OK
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

total_samples = len(Y_train)
weight_class_0 = total_samples / np.sum(Y_train == 0)  # Weight for class 0
weight_class_1 = total_samples / np.sum(Y_train == 1)  # Weight for class 1

RF_algo = RF()
RF_model = RF_algo.fit(X_train, Y_train)

GBDT_algo = GBDT()
GBDT_model = GBDT_algo.fit(X_train, Y_train)

XGB_algo = XGB()
XGB_model = XGB_algo.fit(X_train, Y_train)


from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform, randint
from sklearn.utils.class_weight import compute_sample_weight

def random_search(algo, hyperparameters, X_train, Y_train):
  # do the search using 5 folds/chunks
  clf = RandomizedSearchCV(algo, hyperparameters, cv=5, random_state=2015,
                          scoring='precision_macro', n_iter=20, refit=True)

  # pass the data to fit/train
  clf.fit(X_train, Y_train)

  return clf.best_params_

# RF
RF_tuned_parameters = {
    'n_estimators': randint(50, 500), # Draw from a uniform distribution between 50 and 500
    'max_depth': randint(2, 7),  # Draw from a uniform distribution between 2 and 7
    'min_samples_split': randint(2, 7),  # Draw from a uniform distribution between 2 and 7
    'max_features': ['sqrt', 'log2', None]
}

RF_best_params = random_search(RF(class_weight='balanced'), RF_tuned_parameters, X_train, Y_train)

# GBDT
GBDT_tuned_parameters = {
    'n_estimators': randint(25, 250), # Draw from a uniform distribution between 50 and 500
    'learning_rate': uniform(loc=0.01, scale=4.99),  # Draw from a uniform distribution between 0.01 and 5
    'criterion': ['friedman_mse', 'squared_error'],
    'max_depth': randint(2, 7)  # Draw from a uniform distribution between 2 and 7
}

sample_weights = compute_sample_weight(class_weight='balanced', y=Y_train)
GBDT_best_params = random_search(GBDT(), GBDT_tuned_parameters, X_train, Y_train)


# XGBDT
XGB_tuned_parameters = {
    'n_estimators': randint(25, 250), # Draw from a uniform distribution between 50 and 500
    # eta is learning rate
    'eta': uniform(loc=0.01, scale=4.99),  # Draw from a uniform distribution between 0.01 and 5
    # objective is the same as criterion
    'objective': ['binary:logistic', 'binary:hinge'],
    'max_depth': randint(2, 7)  # Draw from a uniform distribution between 2 and 7
}

scale_pos_weight = np.sum(Y_train == 0)/np.sum(Y_train == 1)
XGB_best_params = random_search(XGB(scale_pos_weight=scale_pos_weight), XGB_tuned_parameters, X_train, Y_train)


RF_algo = RF(**RF_best_params, class_weight= 'balanced')
RF_model = RF_algo.fit(X_train, Y_train)

GBDT_algo = GBDT(**GBDT_best_params)
GBDT_model = GBDT_algo.fit(X_train, Y_train, sample_weight=sample_weights)

XGB_algo = XGB(**XGB_best_params, scale_pos_weight = scale_pos_weight)
XGB_model = XGB_algo.fit(X_train, Y_train)


(61476, 12)
(15369, 12)
(61476,)
(15369,)


# Evaluation

In [ ]:
models = [RF_model, GBDT_model, XGB_model]
names = ['Random Forest', 'GBDT', 'XGBDT']

for i in range(3):
  print(f"Model: {names[i]}")

  # predict based on training data
  predict = models[i].predict(X_train)

  # Calculate precision, recall, and F1-score
  precision, recall, f1_score, _ = precision_recall_fscore_support(Y_train, predict, average='macro')
  print(f"Macro Precision: {precision}")
  print(f"Macro Recall: {recall}")
  print(f"Macro F1-score: {f1_score}")
  print("\n")

for i in range(3):
  print(f"Model: {names[i]}")

  # predict based on training data
  predict = models[i].predict(X_train)

  # Calculate precision, recall, and F1-score
  precision, recall, f1_score, _ = precision_recall_fscore_support(Y_train, predict, average='micro')
  print(f"Micro Precision: {precision}")
  print(f"Micro Recall: {recall}")
  print(f"Micro F1-score: {f1_score}")
  print("\n")

for i in range(3):
  print(f"Model: {names[i]}")

  # predict based on training data
  predict = models[i].predict(X_train)

  # Calculate precision, recall, and F1-score
  precision, recall, f1_score, _ = precision_recall_fscore_support(Y_train, predict, average=None)
  print(f"Class 1 Precision: {precision[1]}")
  print(f"Class 1 Recall: {recall[1]}")
  print(f"Class 1 F1-score: {f1_score[1]}")
  print("\n")

  print(f"Class 0 Precision: {precision[0]}")
  print(f"Class 0 Recall: {recall[0]}")
  print(f"Class 0 F1-score: {f1_score[0]}")
  print("\n")


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay as CM

# Random Forest
print("Random Forest Confusion Matrix")
predict = RF_model.predict(X_test)
CM.from_predictions(Y_test, predict)

# GBDT
print("GBDT Confusion Matrix")
predict = GBDT_model.predict(X_test)
print(CM.from_predictions(Y_test, predict))

# XGB
print("XGB Confusion Matrix")
predict = XGB_model.predict(X_test)
print(CM.from_predictions(Y_test, predict))


#%%
#random check

items = items.groupby(['order_id'], as_index=False).agg({
    'price': 'sum',
    'freight_value': 'sum',
    'seller_id': 'first'
})

items['freight_percent'] = (items['freight_value'] / items['price'])


check =basedata2.merge(items, how='left', on='order_id')
f_check2 = check[check['freight_percent']-check['freight ratio'] > 1]

#%%
import time
from sklearn.inspection import permutation_importance

start_time = time.time()
result = permutation_importance(
    RF_algo, X_test, Y_test, n_repeats=10, random_state=42, n_jobs=2
)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

feature_names = x_values.columns.tolist()
forest_importances = pd.Series(result.importances_mean, index=feature_names)

importances = RF_algo.feature_importances_
forest_importances = pd.Series(importances, index=feature_names)
std = np.std([tree.feature_importances_ for tree in RF_algo.estimators_], axis=0)

import matplotlib.pyplot as plt
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()


from xgboost import plot_importance
plot_importance(XGB_algo, max_num_features=10) # top 10 most important features
plt.show()


GBDT_importances = GBDT_algo.feature_importances_
GBDT_importances = pd.Series(GBDT_importances, index=X_train.columns)
#GBDTstd = np.std([tree.feature_importances_ for tree in GBDT_algo.estimators_], axis=0)



fig, ax = plt.subplots(figsize=(10, 6))
GBDT_importances.plot.bar( ax=ax, capsize=4)
ax.set_title("GBDT Feature Importances using MDI")
ax.set_ylabel("Mean Decrease in Impurity")
ax.set_xlabel("Features")
fig.tight_layout()

plt.show()


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(model, X_train, Y_train, X_test, Y_test):
    print(f"{model}")

    # Predict on training data
    train_preds = model.predict(X_train)
    train_precision = precision_score(Y_train, train_preds, average=None)[1]  # Precision for Class 1
    train_recall = recall_score(Y_train, train_preds, average='binary')  # Recall for Class 1 (binary classification)
    train_f1 = f1_score(Y_train, train_preds, average='binary')  # F1 for Class 1 (binary classification)

    # Predict on test data
    test_preds = model.predict(X_test)
    test_precision = precision_score(Y_test, test_preds, average=None)[1]  # Precision for Class 1
    test_recall = recall_score(Y_test, test_preds, average='binary')  # Recall for Class 1 (binary classification)
    test_f1 = f1_score(Y_test, test_preds, average='binary')  # F1 for Class 1 (binary classification)

    # Print results
    print(f"Training - Precision (Class 1): {train_precision:.4f}, Recall (Class 1): {train_recall:.4f}, F1 (Class 1): {train_f1:.4f}")
    print(f"Test     - Precision (Class 1): {test_precision:.4f}, Recall (Class 1): {test_recall:.4f}, F1 (Class 1): {test_f1:.4f}")
    print("\n")

# Evaluate all models
models = [RF_model, GBDT_model, XGB_model]
for model in models:
    evaluate_model(model, X_train, Y_train, X_test, Y_test)